In [ ]:
import requests
import re
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
import json
import time

In [ ]:
# This file is for crawling indiankanoon and collecting all links

In [ ]:
URL = 'https://indiankanoon.org/browse/' # url of site from where we are scraping
page = s.get(URL) # sending a request to the url
soup = bs(page.content, 'html.parser')
print(page.status_code)
results = soup.find_all('div',{'class':'browselist'})
results = list(results[0:1])
links = {}
no_of_pages = list(range(100))
base = 'https://indiankanoon.org'
f=open('SC_links.txt','w')
for link in results: # loop for multiple courts, if scraping data from multiple courts
    print('start')
    linkd = link.find('a')['href']
    court_name = link.find('a').text
    URL = base+linkd
    page = s.get(URL)
    soup = bs(page.content, 'html.parser')
    result_new = soup.find_all('div',{'class': 'browselist'})
    links[court_name] = []
    for link_new in result_new: # loop for every year
        print((link_new.find('a').text) + " Year Started .....\n")
        if((int)(link_new.find('a').text) < 1980 or (int)(link_new.find('a').text) > 1990):
            continue
        URL = base + link_new.find('a')['href']
        page = s.get(URL)
        soup = bs(page.content, 'html.parser')
        result_new2 = soup.find_all('div',{'class':'browselist'})
        for link_new2 in result_new2: # loop for every month
            for page_in in no_of_pages: # loop for every page
                URL = base + link_new2.find('a')['href']
                URL = URL + '&pagenum={}'.format(page_in)
                URL=URL.replace(' ','%20')
                page = s.get(URL)
                soup = bs(page.content, 'html.parser')
                result_new3=soup.find_all('a',{'class':'cite_tag'})   #<--- I don't know but they changed class I guess, so you have to access it like this now
                for i in result_new3:
                    if i.text.lower()=='full document':
                        URL=base+i['href']
                        f.write(URL+'\n')
                        links[court_name].append(URL)
                if(len(result_new3) == 0):
                    break

        print("Current Year Completed\n")

f.close()
valid_court_name = ['Supreme Court of India']

final_list = {}
for court_name in valid_court_name:
  final_list[court_name] = links[court_name]

json_object = json.dumps(final_list, indent = 4) 

# saving the dictionary with links in a json file
with open("links_Supreme_Court.json", "w") as outfile:  
    outfile.write(json_object)